In [389]:
from selenium import webdriver
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
import random
from datetime import datetime, timedelta
import pickle
from fuzzywuzzy import fuzz, process
from fractions import Fraction
import time

In [155]:
def getname(name, l):
    if len(l) == 0:
        return ''
    highest = process.extractOne(name,l)
    return highest[0]

In [241]:
global oddschecker
oddschecker = 'https://www.oddschecker.com/football/'

In [426]:
def get_stats(driver, url):
    #add time.sleep maybe wait condition
    driver.implicitly_wait(random.randint(0, 6))
    driver.get(url)
    ht = driver.find_elements(By.XPATH, '//span[@class="css-8ngw4i-bottomRow"]')[1].text
    if ht != 'Half-Time':
        time.sleep(3 * 60)
        ht = driver.find_elements(By.XPATH, '//span[@class="css-8ngw4i-bottomRow"]')[1].text
    button = driver.find_elements(By.XPATH, '(//button[@class="css-12l9xvx-ExpandButton e7i6a0l0"])[2]')[0]
    button.click()
    div1 = driver.find_elements(By.XPATH, '//li[@class="css-radwzz-Stat e683amr6"]')
    t = {k[1]: [k[0], k[2]] for k in [i.split('\n') for i in [x for x in [d.text for d in div1]
          if x not in ['Shots','Passes','Expected goals (xG)','Discipline','Defence','Duels']]]}
    score = driver.find_element(By.XPATH, '//span[@class="css-bw7eig-topRow"]').get_attribute('textContent').split(' ')
    t['score'] = [score[0], score[2]]
    posession = driver.find_element(By.XPATH, '//div[@class="css-7s52se-PossessionWheel e683amr3"]').text.split('\n')
    t['posession'] = [posession[1].replace('%',''), posession[2].replace('%','')]
    game_stats = {}
    for name, stats in t.items():
        if '(' in ''.join(stats):
            s = stats[0].split(' ') + stats[1].split(' ')
            if len(s) == 4:
                s = [i.replace('(', '').replace(')','').replace('%','') for i in s]
                game_stats[name] = [s[0], s[2]]
                game_stats[name + ' percentage'] = [int(s[1])/100, int(s[3])/100]
            else:
                s = [i for i in s if '%' not in i]
                game_stats[name] = s
        else:
            game_stats[name] = stats
    game_stats = {k: float(v[0]) - float(v[1]) for k, v in game_stats.items()}
    game_stats['match_name'] = url.split('/')[-1]
    game_stats['half_time'] = ht
    return game_stats

In [398]:
get_stats(webdriver.Chrome(), 'https://www.fotmob.com/match/3888506/matchfacts/toluca-vs-puebla')

{'Expected goals (xG)': 0.25,
 'Total shots': 4.0,
 'Big chances': 0.0,
 'Big chances missed': 1.0,
 'Accurate passes': 108.0,
 'Accurate passes percentage': 0.07999999999999996,
 'Fouls committed': -3.0,
 'Offsides': 0.0,
 'Corners': 2.0,
 'Shots off target': -2.0,
 'Shots on target': 1.0,
 'Blocked shots': 5.0,
 'Hit woodwork': 0.0,
 'Shots inside box': 6.0,
 'Shots outside box': -2.0,
 'xG open play': -0.17000000000000004,
 'xG set play': 0.42,
 'xG on target (xGOT)': -0.55,
 'Passes': 115.0,
 'Own half': 55.0,
 'Opposition half': 53.0,
 'Accurate long balls': 10.0,
 'Accurate long balls percentage': 0.22999999999999993,
 'Accurate crosses': 1.0,
 'Accurate crosses percentage': 0.15000000000000002,
 'Throws': 4.0,
 'Yellow cards': -2.0,
 'Red cards': 0.0,
 'Tackles won': -1.0,
 'Tackles won percentage': -0.24000000000000005,
 'Interceptions': -3.0,
 'Blocks': -2.0,
 'Clearances': 3.0,
 'Keeper saves': -2.0,
 'Duels won': 7.0,
 'Ground duels won': 3.0,
 'Ground duels won percentage':

In [399]:
xg_leagues = {'UNITED STATES - MAJOR LEAGUE SOCCER':'world/usa/mls','ENGLAND - CHAMPIONSHIP':'english/championship',
              'ENGLAND - PREMIER LEAGUE': 'english/premier-league', 'GERMANY - BUNDESLIGA': 'germany/bundesliga',
              'FRANCE - LIGUE 1': 'france/ligue-1','AUSTRIA - BUNDESLIGA': 'austria/bundesliga',
              'PORTUGAL - LIGA PORTUGAL':'portugal/primeira-liga', 'BELGIUM - FIRST DIVISION A':'belgium/jupiler-pro-league',
              'SWITZERLAND - SUPER LEAGUE':'switzerland/super-league','SPAIN - LALIGA':'spain/la-liga-primera',
             'DENMARK - SUPERLIGAEN':'denmark/superligaen', 'MEXICO - LIGA MX APERTURA':'world/mexico/liga-mx',
             'NETHERLANDS - EREDIVISIE':'netherlands/eredivisie'}

def fotmob_game_data(driver, url):
    driver.implicitly_wait(3)
    driver.get(url)
    divs = driver.find_elements(By.XPATH, '//div[@class="css-1akrvvq-Group e7pc1841"]')
    today = url[-8:]
    league = {}
    for count, bucket in enumerate(divs):
        league_name = bucket.find_elements(By.XPATH, '//a[@class="css-vfisw6-GroupTitleLink e7pc1842"]')[count].text
        if league_name not in xg_leagues.keys():
            continue
        url = bucket.find_elements(By.TAG_NAME, 'a')
        urls = [u.get_attribute('href') for u in url]
        time = bucket.text
        time = [datetime.strptime(i + today, '%H:%M%Y%m%d') + timedelta(minutes=53) for i in time.split('\n') if ':' in i]
        urls = urls[1:]
        names = [u.split('/')[-1] for u in urls]
        league[league_name] = {n: [u,t] for u, t, n in zip(urls, time, names)}
    return league

In [402]:
game_data = fotmob_game_data(webdriver.Chrome(), 'https://www.fotmob.com/?filter=&show=all&q=&date=20220803')

In [403]:
game_data

{'UNITED STATES - MAJOR LEAGUE SOCCER': {'charlotte-fc-vs-dc-united': ['https://www.fotmob.com/match/3787562/matchfacts/charlotte-fc-vs-dc-united',
   datetime.datetime(2022, 8, 3, 19, 53)],
  'columbus-crew-vs-cf-montreal': ['https://www.fotmob.com/match/3787700/matchfacts/columbus-crew-vs-cf-montreal',
   datetime.datetime(2022, 8, 3, 20, 23)],
  'portland-timbers-vs-nashville-sc': ['https://www.fotmob.com/match/3787560/matchfacts/portland-timbers-vs-nashville-sc',
   datetime.datetime(2022, 8, 3, 23, 23)],
  'san-jose-earthquakes-vs-inter-miami-cf': ['https://www.fotmob.com/match/3787559/matchfacts/san-jose-earthquakes-vs-inter-miami-cf',
   datetime.datetime(2022, 8, 3, 23, 23)]},
 'MEXICO - LIGA MX APERTURA': {'fc-juarez-vs-atletico-de-san-luis': ['https://www.fotmob.com/match/3888507/matchfacts/fc-juarez-vs-atletico-de-san-luis',
   datetime.datetime(2022, 8, 3, 20, 53)]}}

In [246]:
def get_all_odds_links(driver):
    odds_links = {}
    for f_league, url_league in xg_leagues.items():
        driver = webdriver.Chrome()
        driver.implicitly_wait(2)
        url = oddschecker + url_league
        driver.get(url)
        divs = driver.find_elements(By.XPATH, '//a[@class="beta-callout full-height-link whole-row-link"]')
        mt = []
        for div in divs:
            odds_url = div.get_attribute('href')
            mt.append(odds_url)
        odds_links[f_league] = mt
    return odds_links

In [404]:
all_odds = get_all_odds_links(webdriver.Chrome())

In [405]:
def connect_links(data, all_odds):
    for l in data.keys():
        fotmob_names = data[l].keys()
        odds_names = {v: k for k, v in zip(all_odds[l],[str(i).split('/')[-2] for i in all_odds[l]])}
        for game in fotmob_names:
            match = getname(game, odds_names.keys())
            try:
                link = odds_names[match]
            except:
                link = ''
            data[l][game] = data[l][game] + [link]
    scraper_data = {}
    for l_name, data in data.items():
        for game_name, l1_t_l2 in data.items():
            scraper_data[(game_name, l1_t_l2[1])] = [l_name, l1_t_l2[0], l1_t_l2[2]]
    return scraper_data

In [406]:
days_data = connect_links(game_data, all_odds)

In [407]:
days_data

{('charlotte-fc-vs-dc-united',
  datetime.datetime(2022, 8, 3, 19, 53)): ['UNITED STATES - MAJOR LEAGUE SOCCER', 'https://www.fotmob.com/match/3787562/matchfacts/charlotte-fc-vs-dc-united', 'https://www.oddschecker.com/football/world/usa/mls/charlotte-fc-v-dc-united/winner'],
 ('columbus-crew-vs-cf-montreal',
  datetime.datetime(2022, 8, 3, 20, 23)): ['UNITED STATES - MAJOR LEAGUE SOCCER', 'https://www.fotmob.com/match/3787700/matchfacts/columbus-crew-vs-cf-montreal', 'https://www.oddschecker.com/football/world/usa/mls/columbus-crew-v-cf-montreal/winner'],
 ('portland-timbers-vs-nashville-sc',
  datetime.datetime(2022, 8, 3, 23, 23)): ['UNITED STATES - MAJOR LEAGUE SOCCER', 'https://www.fotmob.com/match/3787560/matchfacts/portland-timbers-vs-nashville-sc', 'https://www.oddschecker.com/football/world/usa/mls/portland-timbers-v-nashville-sc/winner'],
 ('san-jose-earthquakes-vs-inter-miami-cf',
  datetime.datetime(2022, 8, 3, 23, 23)): ['UNITED STATES - MAJOR LEAGUE SOCCER',
  'https://ww

In [392]:
def get_odds(driver, url):
    driver.implicitly_wait(2)
    url = url
    driver.get(url)
    odds = driver.find_element(By.XPATH, '//div[@class="ExpanderBestOddsWrapper_e1jc6hcf"]').text.split('\n')
    odds = [float(Fraction(j)) + 1.0 if '/' in j else j for j in odds]
    odds = dict(zip(odds[::2], odds[1::2]))
    return odds

In [393]:
get_odds(webdriver.Chrome(), 'https://www.oddschecker.com/football/world/usa/mls/columbus-crew-v-cf-montreal/winner')

{'Columbus Crew': 2.0625, 'Draw': 3.6470588235294117, 'CF Montreal': 3.8}

In [414]:
days_data[('charlotte-fc-vs-dc-united', datetime(2022, 8, 3, 19, 53))][2]

'https://www.oddschecker.com/football/world/usa/mls/charlotte-fc-v-dc-united/winner'

In [ ]:
1 + 1

In [423]:
while True:
    for tup in days_data.keys():
        if tup[1] <= datetime.now():
            stats = get_stats(webdriver.Chrome(), days_data[tup][1])
            odds = get_odds(webdriver.Chrome(), days_data[tup][2])
            row = {**stats,**odds}
            row['league'] = days_data[tup][0]
            print(row)
        else:
            print(f'{datetime.now().strftime("%m/%d/%Y, %H:%M:%S")} is not a halftime')
    time.sleep(4 * 60)

08/02/2022, 22:25:33 is not halftime
08/02/2022, 22:25:33 is not halftime
08/02/2022, 22:25:33 is not halftime
08/02/2022, 22:25:33 is not halftime
08/02/2022, 22:25:33 is not halftime


KeyboardInterrupt: 